# Integrated project: Validating our data

In this project I am diving into the agricultural dataset again to continue to validate our data. First I build a data pipeline that will ingest and clean our data with the press of a button, cleaning up our code significantly. 

First I re-organise or refactor our code into modules. I reorganise all of the code into smaller modules so the code is more readable, maintainable and extendable.

I create a module to interact with the database, a module to transform and clean the field-related data and another module to process the weather data.

This makes our code more modular. If we want to debug a problem in the field data, we know where to go, and if we want to import other IoT weather sensors we can just modify the weather data module.

# data_ingestion.py module

This module conntains functions for connecting to a database, and querying some data.

In [1]:
# Importing create_db_engine, query_data and read_from_web_CSV from data_ingestion.py module
#from data_ingestion import create_db_engine, query_data, read_from_web_CSV

# field_data_processor.py module

This module contains a Class that encapsulates the whole data processing process for the field-related data called FieldDataProcessor. In the class, there is a DataFrame attribute and methods that alter that attribute. So I encapsulate all of the logic in this FieldDataProcessor class, I abstract all of the details and only need to call something like FieldDataProcessor.process_data().

Let's test the working of the two modules: 

In [2]:
#Configuration details
config_params = {
    "sql_query": """
        SELECT *
    FROM geographic_features
    LEFT JOIN weather_features USING (Field_ID)
    LEFT JOIN soil_and_crop_features USING (Field_ID)
    LEFT JOIN farm_management_features USING (Field_ID)
            """, # SQL query
    "db_path": 'sqlite:///Maji_Ndogo_farm_survey_small.db', # db_path of the database
    "columns_to_rename": {'Annual_yield': 'Crop_type', 'Crop_type': 'Annual_yield'},# Disctionary of columns we want to swop the names of, 
    "values_to_rename": {'cassaval': 'cassava', 'wheatn': 'wheat', 'teaa': 'tea'}, # Croptype renaming dictionary
    "weather_csv_path": "https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Maji_Ndogo/Weather_station_data.csv", # Weather data CSV here
    "weather_mapping_csv": "https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Maji_Ndogo/Weather_data_field_mapping.csv" # Weather data mapping CSV here
}

In [3]:
#Importing FieldDataProcessor class from field_data_processor.py module
from field_data_processor import FieldDataProcessor

In [4]:
# Instantiating the class with config_params passed to the class as a parameter 
field_processor = FieldDataProcessor(config_params)
field_processor.process()
field_df = field_processor.df

# Test
field_df.head()

2024-02-28 13:07:17,345 - data_ingestion - INFO - Database engine created successfully.
2024-02-28 13:07:17,532 - data_ingestion - INFO - Query executed successfully.
2024-02-28 13:07:17,533 - field_data_processor.FieldDataProcessor - INFO - Sucessfully loaded data.
2024-02-28 13:07:17,542 - field_data_processor.FieldDataProcessor - INFO - Swapped columns: Annual_yield with Crop_type
2024-02-28 13:07:18,487 - data_ingestion - INFO - CSV file read successfully from the web.


,Field_ID,Elevation,Latitude,Longitude,Location,Slope,Rainfall,Min_temperature_C,Max_temperature_C,Ave_temps,Soil_fertility,Soil_type,pH,Pollution_level,Plot_size,Annual_yield,Crop_type,Standard_yield,Unnamed: 0,Weather_station
0,40734,786.05580,-7.389911,-7.556202,Rural_Akatsi,14.795113,1125.2,-3.1,33.1,15.00,0.62,Sandy,6.169393,0.085267,1.3,0.751354,cassava,0.577964,0,4
1,30629,674.33410,-7.736849,-1.051539,Rural_Sokoto,11.374611,1450.7,-3.9,30.6,13.35,0.64,Volcanic,5.676648,0.399684,2.2,1.069865,cassava,0.486302,1,0
2,39924,826.53390,-9.926616,0.115156,Rural_Sokoto,11.339692,2208.9,-1.8,28.4,13.30,0.69,Volcanic,5.331993,0.358029,3.4,2.208801,tea,0.649647,2,0
3,5754,574.94617,-2.420131,-6.592215,Rural_Kilimani,7.109855,328.8,-5.8,32.2,13.20,0.54,Loamy,5.328150,0.286687,2.4,1.277635,cassava,0.532348,3,1
4,14146,886.35300,-3.055434,-7.952609,Rural_Kilimani,55.007656,785.2,-2.5,31.0,14.25,0.72,Sandy,5.721234,0.043190,1.5,0.832614,wheat,0.555076,4,1


# weather_data_processor.py module

 This module contains the WeatherDataProcessor class which deals with all of the weather-related data.

In [5]:
#New configuration details
config_params =  {
    "sql_query": """
        SELECT *
    FROM geographic_features
    LEFT JOIN weather_features USING (Field_ID)
    LEFT JOIN soil_and_crop_features USING (Field_ID)
    LEFT JOIN farm_management_features USING (Field_ID)
            """, # SQL query
    "db_path": 'sqlite:///Maji_Ndogo_farm_survey_small.db', # db_path of the database
    "columns_to_rename": {'Annual_yield': 'Crop_type', 'Crop_type': 'Annual_yield'},# Disctionary of columns we want to swop the names of, 
    "values_to_rename": {'cassaval': 'cassava', 'wheatn': 'wheat', 'teaa': 'tea'}, # Croptype renaming dictionary
    "weather_csv_path": "https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Maji_Ndogo/Weather_station_data.csv", # Weather data CSV here
    "weather_mapping_csv": "https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Maji_Ndogo/Weather_data_field_mapping.csv", # Weather data mapping CSV here


    # Added two new keys
   "weather_csv_path":  "https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Maji_Ndogo/Weather_station_data.csv", # URL for the weather station data
    "regex_patterns" :  {
    'Rainfall': r'(\d+(\.\d+)?)\s?mm',
     'Temperature': r'(\d+(\.\d+)?)\s?C',
    'Pollution_level': r'=\s*(-?\d+(\.\d+)?)|Pollution at \s*(-?\d+(\.\d+)?)'
    } # Regex pattern we use to process the messages
}


In [6]:
# Import weatherDataprocessor class from weather_data_processor.py
from weather_data_processor import WeatherDataProcessor

In [7]:
weather_processor = WeatherDataProcessor(config_params)
weather_processor.process()
weather_df = weather_processor.weather_df

weather_df.head()

2024-02-28 13:07:19,716 - data_ingestion - INFO - CSV file read successfully from the web.
2024-02-28 13:07:19,716 - weather_data_processor.WeatherDataProcessor - INFO - Successfully loaded weather station data from the web.
2024-02-28 13:07:19,761 - weather_data_processor.WeatherDataProcessor - INFO - Messages processed and measurements extracted.
2024-02-28 13:07:19,761 - weather_data_processor.WeatherDataProcessor - INFO - Data processing completed.


,Weather_station_ID,Message,Measurement,Value
0,0,【2022-01-04 21:47:48】温度感应: 现在温度是 12.82C.,Temperature,12.82
1,2,"环境监测报告: 2022-10-25 11:53:44, Air Quality Index...",Pollution_level,0.18
2,0,Temp. Reading [2023-05-23 09:41:36]: Current 1...,Temperature,14.53
3,0,"Weather Update: As of 2022-08-29 06:44:16, rai...",Rainfall,1917.49
4,4,Temp. Reading [2023-10-23 12:48:18]: Current 1...,Temperature,13.23


### Validating our data pipeline

Before analysing the data, let me pause for a second and ask: Did the changes actually get applied? Did I correct the elevation data, did I rename the columns? 

To test the data, we have to give pytest access to that data. The simplest way to do this is by creating CSV files, importing them into validate_data.py, and running the tests.

The following code creates CSV files, runs pytest in the terminal using !pytest validate_data.py -v, and deletes the CSV files once the test is complete.

In [8]:
#!pip install pytest

weather_df.to_csv('sampled_weather_df.csv', index=False)
field_df.to_csv('sampled_field_df.csv', index=False)

!pytest validate_data.py -v

import os# Define the file paths
weather_csv_path = 'sampled_weather_df.csv'
field_csv_path = 'sampled_field_df.csv'

# Delete sampled_weather_df.csv if it exists
if os.path.exists(weather_csv_path):
    os.remove(weather_csv_path)
    print(f"Deleted {weather_csv_path}")
else:
    print(f"{weather_csv_path} does not exist.")

# Delete sampled_field_df.csv if it exists
if os.path.exists(field_csv_path):
    os.remove(field_csv_path)
    print(f"Deleted {field_csv_path}")
else:
    print(f"{field_csv_path} does not exist.")

============================= test session starts =============================
platform win32 -- Python 3.9.18, pytest-8.0.2, pluggy-1.4.0 -- C:\Users\hp\.conda\envs\Sqlenv\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\hp\Documents\ALX Data Science\PYTHON\MY PROJECTS
plugins: anyio-3.5.0
collecting ... collected 7 items

validate_data.py::TestDataValidation::test_crop_types_are_valid PASSED   [ 14%]
validate_data.py::TestDataValidation::test_field_DataFrame_columns PASSED [ 28%]
validate_data.py::TestDataValidation::test_field_DataFrame_non_negative_elevation PASSED [ 42%]
validate_data.py::TestDataValidation::test_positive_rainfall_values PASSED [ 57%]
validate_data.py::TestDataValidation::test_read_field_DataFrame_shape PASSED [ 71%]
validate_data.py::TestDataValidation::test_read_weather_DataFrame_shape PASSED [ 85%]
validate_data.py::TestDataValidation::test_weather_DataFrame_columns PASSED [100%]

============================== 7 passed in 7.61s ===========================

Now we know our data resembles what we expect! 

## Hypothesis

So what are we testing with our null hypothesis $H_0$? Well, we want to know if our field data is representing the reality in Maji Ndogo by looking at an independent set of data. If our field data (means) are the same as the weather data (means), then it indicates no significant difference between the datasets. We're essentially saying that any difference we see between these means is because of randomness. However, if the means differ significantly, we'll know there is a reason for it, and that it is not just a random fluctuation in the data. 

<br>

Given a significance level $\alpha$ of 0.05 for a two-tailed test, we have the following conditions for our hypothesis test at a 95% confidence interval:

- $H_0$: There is no significant difference between the means of the two datasets. This is expressed as $\mu_{field} = \mu_{weather}$.

- $H_a$: There is a significant difference between the means of the two datasets. This is expressed as $\mu_{field} \neq \mu_{weather}$.

<br>

If the p-value obtained from the test:
- is less than or equal to the significance level, so $p \leq \alpha$, we reject the null hypothesis.
- is larger than the significance level, so $p > \alpha$, we cannot reject the null hypothesis, as we cannot find a statistically significant difference between the datasets at the 95% confidence level.

We import all of the packages and define a few variables. 
`.ttest_ind()` takes in two data columns and calculates means, variance, and returns the the t- and p-statistics. Since our alternative hypothesis does not make a claim of greater or less than, we will use the two-sided t-test, by adding  the `alternative = 'two-sided'` keyword.

In [9]:
from scipy.stats import ttest_ind
import numpy as np

# Now, the measurements_to_compare can directly use 'Temperature', 'Rainfall', and 'Pollution_level'
measurements_to_compare = ['Temperature', 'Rainfall', 'Pollution_level']

We want to compare the means of the temperature, rainfall, and pollution data, for fields assigned to a specific weather station. So for both datasets, we need to isolate the measurement type and weather station for each data, so we're comparing the correct means.

Let's break down what  do:
1. We need to filter both `field_df` and `weather_df` based on the given station ID and measurement. We can use `filter_field_data(df, station_id, measurement)` and `filter_weather_data(df, station_id, measurement)`.  
2. We need to perform a t-test to conduct the t-test on the filtered data. So we're going to use `ttest_ind(data_col1, data_col2, equal_var=False)` from `scipy.stats`.
3. `print_ttest_results(station_id, measurement, p_val, alpha)` to interpret and print the results from the t-test.

We'll first define these functions, focusing on `Temperature` for `station ID = 0`. Then, we'll integrate these functions into a loop that iterates over each station ID and measurement type.

In [10]:
def filter_field_data(df, station_id, measurement):
    """
    Filter field data based on a specific weather station and measurement type.

    Parameters:
    - df (DataFrame): The DataFrame containing field data.
    - station_id (int): The ID of the weather station for which data should be filtered.
    - measurement (str): The type of measurement to filter (e.g., 'Temperature', 'Rainfall').

    Returns:
    - Series: A series containing the filtered data for the specified weather station and measurement.
    """
    station_data = df[df['Weather_station'] == station_id]

    return station_data[ measurement]

In [11]:
# Rename 'Ave_temps' in field_df to 'Temperature' to match weather_df
field_df.rename(columns={'Ave_temps': 'Temperature'}, inplace=True)
field_df.head()

,Field_ID,Elevation,Latitude,Longitude,Location,Slope,Rainfall,Min_temperature_C,Max_temperature_C,Temperature,Soil_fertility,Soil_type,pH,Pollution_level,Plot_size,Annual_yield,Crop_type,Standard_yield,Unnamed: 0,Weather_station
0,40734,786.05580,-7.389911,-7.556202,Rural_Akatsi,14.795113,1125.2,-3.1,33.1,15.00,0.62,Sandy,6.169393,0.085267,1.3,0.751354,cassava,0.577964,0,4
1,30629,674.33410,-7.736849,-1.051539,Rural_Sokoto,11.374611,1450.7,-3.9,30.6,13.35,0.64,Volcanic,5.676648,0.399684,2.2,1.069865,cassava,0.486302,1,0
2,39924,826.53390,-9.926616,0.115156,Rural_Sokoto,11.339692,2208.9,-1.8,28.4,13.30,0.69,Volcanic,5.331993,0.358029,3.4,2.208801,tea,0.649647,2,0
3,5754,574.94617,-2.420131,-6.592215,Rural_Kilimani,7.109855,328.8,-5.8,32.2,13.20,0.54,Loamy,5.328150,0.286687,2.4,1.277635,cassava,0.532348,3,1
4,14146,886.35300,-3.055434,-7.952609,Rural_Kilimani,55.007656,785.2,-2.5,31.0,14.25,0.72,Sandy,5.721234,0.043190,1.5,0.832614,wheat,0.555076,4,1


In [12]:
# Example for station ID 0 and Temperature
station_id = 0
measurement = 'Temperature'


# Filter data for the specific station and measurement
field_values = filter_field_data(field_df, station_id, measurement)
field_values

1       13.35
2       13.30
8       12.80
10      13.70
14      13.35
        ...  
5627    13.30
5630    14.25
5632    11.00
5638    13.30
5642    12.85
Name: Temperature, Length: 1375, dtype: float64

Below I create a data filter function that takes in the weather_df DataFrame, the station_id, and measurement type, and returns a single column (series) of data filtered by the station_id, and measurement.

In [13]:
def filter_weather_data(df, station_id, measurement):
    """
    Filter weather data based on a specific weather station and measurement type.

    Parameters:
    - df (DataFrame): The DataFrame containing weather data.
    - station_id (int): The ID of the weather station for which data should be filtered.
    - measurement (str): The type of measurement to filter (e.g., 'Temperature', 'Rainfall').

    Returns:
    - Series: A series containing the filtered data for the specified weather station and measurement.
    """
    station_data = df[df['Weather_station_ID'] == station_id]
    station_data = station_data[station_data['Measurement'] == measurement]

    return station_data['Value']

In [14]:
# Example for station ID 0 and Temperature
station_id = 0
measurement = 'Temperature'

# Filter data for the specific station and measurement

weather_values = filter_weather_data(weather_df, station_id, measurement)
weather_values

0       12.82
2       14.53
29      14.28
32      12.87
67      13.13
        ...  
1804    12.77
1805    14.13
1817    13.14
1833    14.14
1834    13.61
Name: Value, Length: 100, dtype: float64

Now we create a function that calculates the t-statistic and p-value. The function accepts two single columns of data and return a tuple of the t-statistic and p-value.

In [15]:
def run_ttest(Column_A, Column_B):
    """
    Perform an independent two-sample t-test on two columns of data.

    Parameters:
    - Column_A (array-like): The data values for the first sample.
    - Column_B (array-like): The data values for the second sample.

    Returns:
    - tuple: A tuple containing the t-statistic and p-value from the t-test.
    """
    t_statistic, p_value = ttest_ind(Column_A, Column_B, equal_var=False)
    
    return t_statistic, p_value

In [16]:
# Example for station ID 0 and Temperature
station_id = 0
measurement = 'Temperature'

# Filter data for the specific station and measurement
field_values = filter_field_data(field_df, station_id, measurement)
weather_values = filter_weather_data(weather_df, station_id, measurement)

# Perform t-test
t_stat, p_val = run_ttest(field_values, weather_values)
print(f"T-stat: {t_stat:.5f}, p-value: {p_val:.5f}")

T-stat: -0.11632, p-value: 0.90761


The function below prints out our ttest results.

In [17]:
def print_ttest_results(station_id, measurement, p_val, alpha):
    """
    Interprets and prints the results of a t-test based on the p-value.
    """
    if p_val < alpha:
        print(f"   Significant difference in {measurement} detected at Station  {station_id}, (P-Value: {p_val:.5f} < {alpha}). Null hypothesis rejected.")
    else:
        print(f"   No significant difference in {measurement} detected at Station  {station_id}, (P-Value: {p_val:.5f} > {alpha}). Null hypothesis not rejected.")

In [18]:
# Example for station ID 0 and Temperature
station_id = 0
alpha = 0.05

measurement = 'Temperature'

# Filter data for the specific station and measurement
field_values = filter_field_data(field_df, station_id, measurement)
weather_values = filter_weather_data(weather_df, station_id, measurement)

# Perform t-test
t_stat, p_val = run_ttest(field_values, weather_values)
print_ttest_results(station_id, measurement, p_val, alpha)

   No significant difference in Temperature detected at Station  0, (P-Value: 0.90761 > 0.05). Null hypothesis not rejected.


Now I put it all together in a loop. The function loops over measurements_to_compare and all station_id, performs a t-test and print the results. The function accepts field_df, weather_df, list_measurements_to_compare, alpha. 
The value of alpha should default to a value of 0.05. 

In [19]:
def hypothesis_results(field_df, weather_df, list_measurements_to_compare, alpha = 0.05):
    """
    Perform t-tests for each combination of measurement and weather station and print the results.

    Parameters:
    - field_df (DataFrame): DataFrame containing field data.
    - weather_df (DataFrame): DataFrame containing weather data.
    - list_measurements_to_compare (list): List of measurements to compare.
    - alpha (float, optional): Significance level for hypothesis testing. Defaults to 0.05.

    Returns:
    - None: The function prints the results of t-tests for each combination.
    """
    for station_id in sorted(list(weather_df['Weather_station_ID'].unique())):
        for measurement in list_measurements_to_compare:
            field_values = filter_field_data(field_df, station_id, measurement)
            weather_values = filter_weather_data(weather_df, station_id, measurement)

            t_statistic, p_value = run_ttest(field_values, weather_values)

            print_ttest_results(station_id, measurement, p_value, alpha)

In [20]:
alpha = 0.05
hypothesis_results(field_df, weather_df, measurements_to_compare, alpha)

   No significant difference in Temperature detected at Station  0, (P-Value: 0.90761 > 0.05). Null hypothesis not rejected.
   No significant difference in Rainfall detected at Station  0, (P-Value: 0.21621 > 0.05). Null hypothesis not rejected.
   No significant difference in Pollution_level detected at Station  0, (P-Value: 0.56418 > 0.05). Null hypothesis not rejected.
   No significant difference in Temperature detected at Station  1, (P-Value: 0.47241 > 0.05). Null hypothesis not rejected.
   No significant difference in Rainfall detected at Station  1, (P-Value: 0.54499 > 0.05). Null hypothesis not rejected.
   No significant difference in Pollution_level detected at Station  1, (P-Value: 0.24410 > 0.05). Null hypothesis not rejected.
   No significant difference in Temperature detected at Station  2, (P-Value: 0.88671 > 0.05). Null hypothesis not rejected.
   No significant difference in Rainfall detected at Station  2, (P-Value: 0.36466 > 0.05). Null hypothesis not rejected.
 

For all of our measurements the p-value > alpha, so there is not enough evidence to reject the null hypothesis. This means we have no evidence to suggest that the weather data is different from the field data. This makes us confident that our field data, at least in terms of temperature, rainfall, and pollution level is reflecting the reality. 